In [ ]:
from transformers import HubertPreTrainedModel, BertPreTrainedModel
from transformers.models.hubert.modeling_hubert import HubertFeatureEncoder
from transformers.models.bert.modeling_bert import BertPooler

In [ ]:
class Hubert(HubertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.feature_projection = HubertFeatureEncoder(config)
        # self.feature_projection = HubertFeatureProjection(config)
        # self.encoder = HubertEncoder(config)
        
        self.post_init()
        
    def forward(
        self,
        input_values,
        attention_mask = None,
        mask_time_indices = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        extract_features = self.feature_extractor(input_values)
        extract_features = extract_features.transpose(1, 2)

        if attention_mask is not None:
            # compute reduced attention_mask corresponding to feature vectors
            attention_mask = self._get_feature_vector_attention_mask(extract_features.shape[1], attention_mask)

        hidden_states = self.feature_projection(extract_features)
        hidden_states = self._mask_hidden_states(hidden_states, mask_time_indices=mask_time_indices)

        encoder_outputs = self.encoder(
            hidden_states,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = encoder_outputs[0]

        if not return_dict:
            return (hidden_states,) + encoder_outputs[1:]

In [ ]:
projector = Hubert.from_pretrained('ntu-spml/distilhubert')

In [ ]:
class Pooler(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.pooler = BertPooler(config)
        
        self.post_init()
        
    def forward(self, hidden_states):
        return self.pooler(hidden_states)

In [ ]:
pooler = Pooler.from_pretrained('google/bert_uncased_L-2_H-128_A-2')